In [8]:
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/marcos/computacion/metodos_numericos_2c_2024/film-genre-classifier/film-classifier/")
import knn

df = pd.read_csv("../data/raw/wiki_movie_plots_deduped_sample.csv")

In [3]:
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,tokens,split
0,2009,A Very Short Life,Hong Kong,Dennis Law,"Dennis Law (director); Leila Tong, Maggie Siu,...",crime,https://en.wikipedia.org/wiki/A_Very_Short_Life,"In Hong Kong, Dennis Law, a property developer...",hong kong property developer filmmaker attend ...,train
1,1991,Let Him Have It,British,Peter Medak,"Christopher Eccleston, Tom Courtenay",crime,https://en.wikipedia.org/wiki/Let_Him_Have_It,"Derek Bentley (Eccleston) is an illiterate, ep...",illiterate epileptic young adult developmental...,train
2,2002,Birthday Girl,American,Jez Butterworth,"Nicole Kidman, Ben Chaplin, Vincent Cassel",crime,https://en.wikipedia.org/wiki/Birthday_Girl,"John Buckingham (Ben Chaplin), a lonely St Alb...",lonely st albans bank clerk order mail order b...,train
3,1958,The Man Inside,British,John Gilling,"Jack Palance, Anita Ekberg",crime,https://en.wikipedia.org/wiki/The_Man_Inside_(...,Sam Carter (Patrick) is a jeweler's clerk who ...,jeweler clerk dream steal fortune diamond even...,train
4,2011,X,Australian,Jon Hewitt,Viva Bianca,crime,https://en.wikipedia.org/wiki/X_(2011_film),"Shay (Hanna Mangan-Lawrence), a runaway on her...",shay runaway night sydney work prostitute beat...,train
...,...,...,...,...,...,...,...,...,...,...
395,1954,Dawn at Socorro,American,George Sherman,"Rory Calhoun, Piper Laurie, David Brian, Kathl...",western,https://en.wikipedia.org/wiki/Dawn_at_Socorro,A retired gunfighter (Rory Calhoun) and a salo...,retired gunfighter saloonkeeper play card salo...,train
396,1959,The Miracle of the Hills,American,Paul Landres,"Rex Reason, Betty Lou Gerson",western,https://en.wikipedia.org/wiki/The_Miracle_of_t...,A minister arrives at a run-down mining town t...,minister arrive run mining town church find wo...,test
397,1959,Ride Lonesome,American,Budd Boetticher,Randolph Scott,western,https://en.wikipedia.org/wiki/Ride_Lonesome,Bounty hunter Ben Brigade (Randolph Scott) cap...,bounty hunter capture want outlaw brag brother...,train
398,1952,The Battle at Apache Pass,American,George Sherman,"John Lund, Jeff Chandler",western,https://en.wikipedia.org/wiki/The_Battle_at_Ap...,"In 1862, as the Civil War rages in the Southea...",civil war rage southeastern state southwest fa...,train


In [109]:
print(df.iloc[399]["Genre"])
print()
print(df.iloc[0]["tokens"])

western

hong kong property developer filmmaker attend dinner party host attorney friend party dennis meet female police inspector decide tell police investigation involve murder young girl story flash cat meet police commissioner wish open investigation involve death young girl hand mother join female colleague criminologist ivy police sergeant decide conduct interrogation suspect interrogation becky refuse statement result subject police brutality colleague leave naked cold interrogation room later conduct body cavity search cat later tell involved investigation arrive hospital find little girl lie unconscious body scar die immediately doctor prepare operation later arrest becky boyfriend catch try rid possible evidence computer police find series burn videotape cat reveal tape contain child pornography interrogation reveal officer physically restrain becky force watch videotape feature have sex daughter interrogation flashback reveal heiyi receive multiple scar physically beat whip 

In [5]:
df.value_counts(["Genre","split"])

Genre            split
crime            train    80
romance          train    80
western          train    80
science fiction  train    80
romance          test     20
crime            test     20
science fiction  test     20
western          test     20
Name: count, dtype: int64

In [105]:
# pd.Series(tokens).value_counts().head(10).sort_values().plot(kind='barh', figsize=(10,10));
from variables import *

In [172]:
# string representation of list to list using ast.literal_eval()
tokens = np.hstack(df["tokens"].apply(lambda x: x.split()).values)

len(np.unique(tokens))
unique_tokens = pd.Series(tokens).value_counts().index[:5000].values

unique_tokens_dict = dict(zip(unique_tokens, range(len(unique_tokens))))

X_train = np.zeros((320, len(unique_tokens)), dtype=int)
X_test = np.zeros((80, len(unique_tokens)), dtype=int)
y_train = np.zeros((320, 1), dtype=int)
y_test = np.zeros((80, 1), dtype=int)
itrain = 0
itest = 0

for i, row in df.iterrows():
    if row["split"] == "train":
        for token in row["tokens"].split():
            if unique_tokens_dict.get(token,False)!=False:
                X_train[itrain, unique_tokens_dict[token]] += 1
        y_train[itrain] = genre_name.index(row["Genre"])
        itrain += 1
    else:
        for token in row["tokens"].split():
            if unique_tokens_dict.get(token,False)!=False:
                X_test[itest, unique_tokens_dict[token]] += 1
        y_test[itest] = genre_name.index(row["Genre"])
        itest += 1

In [173]:
print(X[0].shape)
print(df.iloc(0)[124])

(1000,)
Release Year                                                     2008
Title                                                      Chaturanga
Origin/Ethnicity                                              Bengali
Director                                              Suman Mukherjee
Cast                                                              NaN
Genre                                                         romance
Wiki Page             https://en.wikipedia.org/wiki/Chaturanga_(film)
Plot                Based on the novel by author Rabindranath Tago...
tokens              base novel author chaturanga love catch confli...
split                                                            test
Name: 124, dtype: object


In [174]:
# df.info()

In [175]:
train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

print(train_df.shape)
print(test_df.shape)

(320, 10)
(80, 10)


In [176]:
help(knn)

Help on module knn:

NAME
    knn

FUNCTIONS
    calcular_vecinos(X_train, X_dev)

    contar_predicciones_correctas(guesses, y_dev)

    distancia_coseno(dev, train)

    medir_exactitud(vecinos, y_train, y_dev, k) -> float

    moda_cada_elemento(vecinos, y_train, k)

    normalize_data(data)

DATA
    __warningregistry__ = {'version': 671, ('divide by zero encountered in...

FILE
    /home/marcos/computacion/metodos_numericos_2c_2024/film-genre-classifier/film-classifier/knn.py




In [177]:
from knn import *

In [178]:
K = 5
X_train = normalize_data(X_train)
X_test = normalize_data(X_test)
vecinos = calcular_vecinos(X_train, X_test)

promedio_k_fijo = medir_exactitud(vecinos, y_train, y_test, K)

print("Resultados Experimento 2:")
print("Con k: ", K, ", exactitud obtenida: ", promedio_k_fijo, sep = '')



Resultados Experimento 2:
Con k: 5, exactitud obtenida: 0.85
